In [1]:
import dotenv
dotenv.load_dotenv()

True

In [11]:
from langchain.document_loaders import TextLoader

# Load document with LangChain's TextLoader
    
loader = TextLoader('./AI_executive_order_oct_2023.txt')
documents = loader.load()

# Chunk the document

from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 696, which is longer than the specified 600
Created a chunk of size 1186, which is longer than the specified 600
Created a chunk of size 1289, which is longer than the specified 600
Created a chunk of size 988, which is longer than the specified 600
Created a chunk of size 1452, which is longer than the specified 600
Created a chunk of size 1030, which is longer than the specified 600
Created a chunk of size 887, which is longer than the specified 600
Created a chunk of size 1022, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 604, which is longer than the specified 600
Created a chunk of size 1193, which is longer than the specified 600
Created a chunk of size 1015, which is longer than the specified 600
Created a chunk of size 927, which is longer than the specified 600
Created a chunk of size 630, which is longer than the specified 600
Created a chunk of size 785, which is lo

In [12]:
# Create embeddings and vector store

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
    embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

embedded weaviate is already listening on port 6666


{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"langchain_a0689d8fcff84e3094cefc45718e2a8a_PyYEqHYtcKi2","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-12-19T13:30:06-05:00","took":34405}


In [13]:
# Retriever

retriever = vectorstore.as_retriever()

# Augment

from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you do not know.
Use three setences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you do not know.\nUse three setences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n", template_format='f-string', validate_template=True), additional_kwargs={})]


In [14]:
# Generate

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser ()
)

{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2023-12-19T13:30:14-05:00"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:6666","time":"2023-12-19T13:30:14-05:00"}


In [15]:
# Query[0]:

query = "How will the Secretary of Homeland Security create an AI Safety Board?"
rag_chain.invoke(query)

# Output: "The Secretary of Homeland Security will establish an Artificial Intelligence
# Safety and Security Board as an advisory committee. The board will include AI experts
# from the private sector, academia, and government. Its purpose is to provide advice,
# information, or recommendations for improving security, resilience, and incident response
# related to AI usage in critical infrastructure."

Embedded weaviate wasn't listening on port 6666, so starting embedded weaviate again
Started /home/mg/.cache/weaviate-embedded: process ID 522478


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-12-19T13:30:18-05:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-12-19T13:30:18-05:00"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_2f4e7d761bcd4f42ad68821c5a74e599_mqvV0kRdIYTu","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-12-19T13:30:18-05:00","took":4589711}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_8106329845ba457f878de3b20e80eb6d_0CrCTCTejrCQ","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-12-19T13:30:18-05:00","took":3063230}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"langchain_a53b0cd7f66447a89f8e3034832a2da

'The Secretary of Homeland Security will establish an Artificial Intelligence Safety and Security Board as an advisory committee. The board will include AI experts from the private sector, academia, and government. Its purpose is to provide advice, information, or recommendations for improving security, resilience, and incident response related to AI usage in critical infrastructure.'

In [16]:
# Query[1]:

query = "What does the AI Executive Order say about watermarking?"
rag_chain.invoke(query)

# Output: "The AI Executive Order defines watermarking as the act of
# embedding difficult-to-remove information into AI outputs for the
# purpose of verifying authenticity, identity, modifications, or conveyance."

'The AI Executive Order defines watermarking as the act of embedding information into AI outputs for the purpose of verifying authenticity, identity, modifications, or conveyance.'

In [17]:
# Query[2]:

query = "What is the theoretical maximum computing capacity cutoff for the technical conditions for AI models?"
rag_chain.invoke(query)

# Output: "The theoretical maximum computing capacity cutoff for the technical conditions for AI models is 1020
# integer or floating-point operations per second for training AI."
#
# **Note: 1020 means 10^20, and is correct based on the original text document which omits the '^' sign

'The theoretical maximum computing capacity cutoff for the technical conditions for AI models is 1020 integer or floating-point operations per second for training AI.'

In [18]:
# Query[3]:

query = "What is the easiest way to time travel?"
rag_chain.invoke(query)

# Output: "I do not know the answer."

'I do not know the answer.'